In [298]:
import pandas as pd
import os

In [299]:
path = os.getcwd()
parent = os.path.dirname(path)
os.chdir(parent + '/dataset/')

## Coal & Natural Gas Monthly Generation by State

- source file: Net_generation_coal_monthly.csv
    - gives the monthly power generation of each state
- cleaned output file: coal_monthly_generation_by_state.csv

### Coal Monthly Generation

In [301]:
# read the source file

filename = 'Net_generation_coal_monthly.csv'
df = pd.read_csv(filename, header=0)


In [302]:
# select and change column names

us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District Of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
}

rename_mapping = {}

for c in df.columns:
    if ' all sectors ' in c:
        s = c.split(' all sectors ')[0]
        if s in us_state_to_abbrev:
            s = us_state_to_abbrev[s]
            rename_mapping[c] = s

df2 = df[['Month'] + list(rename_mapping.keys())
         ].rename(mapper=rename_mapping, axis=1)

# seperate month and year

month_year = df2['Month'].str.split(
    '-', expand=True).rename({0: 'Month', 1: 'Year'}, axis=1)[['Year', 'Month']]

month_year['Year'] = month_year['Year'].apply(lambda s: '20' + s)

df3 = pd.concat([month_year, df2.drop('Month', axis=1)], axis=1)

# pivot the dataframe

df4 = pd.melt(df3, id_vars=['Year', 'Month']).rename(
    {'variable': 'State', 'value': 'Generation'}, axis=1)
df4['Month'] = pd.Categorical(df4['Month'], ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep',
                                             'Oct', 'Nov', 'Dec'])

# sort the dataframe by year, state, month

df5 = df4.sort_values(by=['Year', 'State', 'Month'])

# select data from 2013 to 2020

years = [str(i) for i in range(2013, 2021)]

df6 = df5[df5['Year'].isin(years)]

df_clean = df6


In [303]:

# save the file

df_clean.to_csv(
    parent + '/dataset_cleaned/coal_monthly_generation_by_state.csv', index=False)


### Natrual Gas Generation

In [304]:
# read the source file

filename = 'Net_generation_natural_gas_monthly.csv'
df = pd.read_csv(filename, header=4)


In [305]:
# select and change column names

rename_mapping = {}

for c in df.columns:
    if ' all sectors ' in c:
        s = c.split(' all sectors ')[0]
        if s in us_state_to_abbrev:
            s = us_state_to_abbrev[s]
            rename_mapping[c] = s

df2 = df[['Month'] + list(rename_mapping.keys())
         ].rename(mapper=rename_mapping, axis=1)

# seperate month and year

month_year = df2['Month'].str.split(
    ' ', expand=True).rename({0: 'Month', 1: 'Year'}, axis=1)[['Year', 'Month']]

df3 = pd.concat([month_year, df2.drop('Month', axis=1)], axis=1)

df4 = pd.melt(df3, id_vars=['Year', 'Month']).rename(
    {'variable': 'State', 'value': 'Generation'}, axis=1)
df4['Month'] = pd.Categorical(df4['Month'], ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep',
                                             'Oct', 'Nov', 'Dec'])

# sort the dataframe by year, state, month

df5 = df4.sort_values(by=['Year', 'State', 'Month'])

# select data from 2013 to 2020

years = [str(i) for i in range(2013, 2021)]

df6 = df5[df5['Year'].isin(years)]

df_clean = df6

In [306]:
# save the file

df_clean.to_csv(
    parent + '/dataset_cleaned/gas_monthly_generation_by_state.csv', index=False)


## Merge Emission Data from 2013 to 2022 

In [321]:
years = list(range(2013, 2021))

frames = []

for y in years: 
    filename = 'yearly_emissions/emissions' + str(y) + '.xlsx'
    
    data_df = pd.read_excel(filename, header=1)
    
    data_df.insert(0, "Year", y)

    frames.append(data_df)
    
emission_df = pd.concat(frames, ignore_index=True).dropna(subset=['Plant Name'])

In [322]:
emission_df

,Year,Plant Code,Plant Name,State,Sector Group,Sector Code,Prime Mover,Fuel Code,Aggregated Fuel Group,Generation (kWh),...,Fuel Consumption for Electric Generation (MMBtu),\n Fuel Consumption for Useful Thermal Output (MMBtu),Quantity of Fuel Consumed,Fuel Units,Tons of CO2 Emissions,Metric Tonnes of CO2 Emissions,NERC Region,Balancing Authority Code,Balancing Authority Name,EIA Balancing Authority Region
0,2013,3,Barry,AL,ELECTRIC POWER,1.0,CA,NG,GAS,2.813479e+09,...,588611.0,0.0,575166,Mcf,3.443374e+04,3.123809e+04,NaN,NaN,NaN,NaN
1,2013,3,Barry,AL,ELECTRIC POWER,1.0,CT,NG,GAS,5.004417e+09,...,54371640.0,0.0,53116114,Mcf,3.180741e+06,2.885549e+06,NaN,NaN,NaN,NaN
2,2013,3,Barry,AL,ELECTRIC POWER,1.0,ST,BIT,COAL,4.962412e+09,...,51202482.0,0.0,2417558,Tons,5.263615e+06,4.775120e+06,NaN,NaN,NaN,NaN
3,2013,3,Barry,AL,ELECTRIC POWER,1.0,ST,NG,GAS,1.168291e+08,...,1206368.0,0.0,1178044,Mcf,7.057253e+04,6.402298e+04,NaN,NaN,NaN,NaN
4,2013,7,Gadsden,AL,ELECTRIC POWER,1.0,ST,BIT,COAL,8.269450e+07,...,1261795.0,0.0,52109,Tons,1.297125e+05,1.176744e+05,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43481,2020,64663,WLWPCF Cogeneration Facility,OR,COMMERCIAL,5.0,IC,NG,GAS,1.062000e+04,...,160.0,912.0,1002.0,Mcf,6.252249e+01,5.672003e+01,WECC,PGE,Portland General Electric Company,Northwest
43482,2020,64748,Fuel Cell 18A Sneden Avenue,NY,ELECTRIC POWER,2.0,FC,NG,GAS,2.335000e+06,...,3637.0,0.0,14549.0,Mcf,2.121216e+02,1.924354e+02,NPCC,NYIS,New York Independent System Operator,New York Independent System Operator
43483,2020,64749,Fuel Cell 18B Sneden Avenue,NY,ELECTRIC POWER,2.0,FC,NG,GAS,2.677000e+06,...,17252.0,0.0,16733.0,Mcf,1.006192e+03,9.128116e+02,NPCC,NYIS,New York Independent System Operator,New York Independent System Operator
43484,2020,64753,FDX010.0 FedEx Fuel Cell,CT,ELECTRIC POWER,2.0,FC,NG,GAS,8.670000e+06,...,60594.0,0.0,58488.0,Mcf,3.534037e+03,3.206058e+03,NPCC,ISNE,New England ISO,New England ISO


In [323]:
# select and rename columns
emission_df2 = emission_df[['Year', 'Plant Code', 'State', 'Fuel Code',
                            'Aggregated Fuel Group', 'Generation (kWh)', 'Tons of CO2 Emissions']]
emission_df2 = emission_df2.rename(
    {'Generation (kWh)': 'Generation', 'Tons of CO2 Emissions': 'Emission'}, axis=1)

# only select COAL
emission_df3 = emission_df2[emission_df2['Aggregated Fuel Group'].isin([
                                                                       'COAL', 'GAS'])]

# aggregate the generation and emissions by year and plant code
emission_df4 = emission_df3.groupby(['Year', 'State', 'Plant Code', 'Aggregated Fuel Group'], as_index=False).aggregate({
    'Generation': 'sum', 'Emission': 'sum'})


In [324]:
emission_df4

,Year,State,Plant Code,Aggregated Fuel Group,Generation,Emission
0,2013,AK,75,GAS,2.216302e+07,2.077440e+04
1,2013,AK,79,COAL,1.878430e+08,3.511266e+05
2,2013,AK,96,GAS,1.379744e+09,8.745762e+05
3,2013,AK,6288,COAL,1.910638e+08,2.658208e+05
4,2013,AK,6292,GAS,4.196500e+07,3.956033e+04
...,...,...,...,...,...,...
20592,2020,WY,56609,COAL,3.203757e+09,3.571382e+06
20593,2020,WY,57703,GAS,3.542980e+08,1.624272e+05
20594,2020,WY,57915,COAL,1.468371e+08,9.347161e+05
20595,2020,WY,57915,GAS,6.580391e+07,2.261733e+05


In [325]:
emission_df4.to_csv(parent + '/dataset_cleaned/yearly_generation_emission.csv', index=False)

## 

## Monthly Generation Data by Power Plant

### 2013

In [312]:
df = pd.read_excel('923/EIA923_Schedules_2_3_4_5_M_12_2013_Final_Revision.xlsx', header = 5)

In [314]:
cols = ['Plant Id', 'Plant Name', 'State', 'Sector Name', 'Reported Fuel Type Code',
        'Netgen_Jan', 'Netgen_Feb', 'Netgen_Mar', 'Netgen_Apr', 'Netgen_May',
        'Netgen_Jun', 'Netgen_Jul', 'Netgen_Aug', 'Netgen_Sep', 'Netgen_Oct',
        'Netgen_Nov', 'Netgen_Dec']

df2 = df[cols]

mapper = {'Plant Id': 'Plant Code', 'Netgen_Jan': 'Jan', 'Netgen_Feb': 'Feb', 'Netgen_Mar': 'Mar',
          'Netgen_Apr': 'Apr', 'Netgen_May': 'May', 'Netgen_Jun': 'Jun', 'Netgen_Jul': 'Jul', 'Netgen_Aug': 'Aug',
          'Netgen_Sep': 'Sep', 'Netgen_Oct': 'Oct', 'Netgen_Nov': 'Nov', 'Netgen_Dec': 'Dec'}

df3 = df2.rename(mapper, axis=1)


def foo(x):
    if x in ['NG']:
        return "GAS"
    elif x in ['SUB', 'WC', 'BIT', 'SGC', 'LIG', 'RC']:
        return "COAL"
    else:
        return 'OTHERS'


df3['Aggregated Fuel Group'] = df3['Reported Fuel Type Code'].apply(foo)

df4 = df3[['Plant Code', 'State', 'Aggregated Fuel Group', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
           'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']]

df5 = df4
for c in ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']:

    df5 = df5[df5[c] != '.']

df6 = df5[df5['Aggregated Fuel Group'].isin(['GAS', 'COAL'])]

df7 = df6.melt(id_vars=['Plant Code', 'State', 'Aggregated Fuel Group']).rename(
    {'variable': 'Month', 'value': 'Generation'}, axis=1)

df8 = df7.groupby(['Plant Code', 'State', 'Aggregated Fuel Group',
                  'Month'], as_index=False).aggregate({'Generation': 'sum'})

df8.insert(0, 'Year', 2013)

_2013 = df8


In [275]:
_2013.head(10)

,Year,Plant Id,State,Aggregated Fuel Group,Month,Generation
0,2013,3,AL,COAL,Apr,244468.68
1,2013,3,AL,COAL,Aug,506485.96
2,2013,3,AL,COAL,Dec,219258.08
3,2013,3,AL,COAL,Feb,334765.21
4,2013,3,AL,COAL,Jan,545082.12
5,2013,3,AL,COAL,Jul,625283.37
6,2013,3,AL,COAL,Jun,672545.09
7,2013,3,AL,COAL,Mar,226058.6
8,2013,3,AL,COAL,May,287540.95
9,2013,3,AL,COAL,Nov,143120.38


### 2014 - 2020

In [315]:

def clean(df):

    cols = ['Plant Id', 'Plant Name', 'Plant State', 'Sector Name', 'Reported\nFuel Type Code',
            'Netgen\nJanuary', 'Netgen\nFebruary', 'Netgen\nMarch', 'Netgen\nApril',
            'Netgen\nMay', 'Netgen\nJune', 'Netgen\nJuly', 'Netgen\nAugust',
            'Netgen\nSeptember', 'Netgen\nOctober', 'Netgen\nNovember',
            'Netgen\nDecember']

    df2 = df[cols]

    mapper = {'Plant Id': 'Plant Code', 'Plant State': 'State', 'Reported\nFuel Type Code': 'Reported Fuel Type Code',
              'Netgen\nJanuary': 'Jan', 'Netgen\nFebruary': 'Feb', 'Netgen\nMarch': 'Mar', 'Netgen\nApril': 'Apr',
              'Netgen\nMay': 'May', 'Netgen\nJune': 'Jun', 'Netgen\nJuly': 'Jul', 'Netgen\nAugust': 'Aug',
              'Netgen\nSeptember': 'Sep', 'Netgen\nOctober': 'Oct', 'Netgen\nNovember': 'Nov',
              'Netgen\nDecember': 'Dec'}

    df3 = df2.rename(mapper, axis=1)

    df3['Aggregated Fuel Group'] = df3['Reported Fuel Type Code'].apply(foo)

    df4 = df3[['Plant Code', 'State', 'Aggregated Fuel Group', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
               'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']]

    df5 = df4
    for c in ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']:

        df5 = df5[df5[c] != '.']

    df6 = df5[df5['Aggregated Fuel Group'].isin(['GAS', 'COAL'])]

    df7 = df6.melt(id_vars=['Plant Code', 'State', 'Aggregated Fuel Group']).rename(
        {'variable': 'Month', 'value': 'Generation'}, axis=1)

    df8 = df7.groupby(['Plant Code', 'State', 'Aggregated Fuel Group',
                       'Month'], as_index=False).aggregate({'Generation': 'sum'})

    return df8


In [316]:
data = [_2013] 
for y in [2014, 2015, 2016, 2017, 2019, 2020]: 
    filename = '923/EIA923_Schedules_2_3_4_5_M_12_' + str(y) + '_Final_Revision.xlsx'
    df = pd.read_excel(filename, header = 5)
    df_clean = clean(df)
    df_clean.insert(0, 'Year', y)
    data.append(df_clean)

In [317]:
output = pd.concat(data)

In [318]:
output.to_csv(parent + '/dataset_cleaned/monthly_generation.csv', index=False)

In [319]:
output

,Year,Plant Code,State,Aggregated Fuel Group,Month,Generation
0,2013,3,AL,COAL,Apr,244468.68
1,2013,3,AL,COAL,Aug,506485.96
2,2013,3,AL,COAL,Dec,219258.08
3,2013,3,AL,COAL,Feb,334765.21
4,2013,3,AL,COAL,Jan,545082.12
...,...,...,...,...,...,...
34351,2020,99999,WI,COAL,Mar,0
34352,2020,99999,WI,COAL,May,0
34353,2020,99999,WI,COAL,Nov,0
34354,2020,99999,WI,COAL,Oct,0
